In this notebook I am going to extract AirBnB data: http://insideairbnb.com/get-the-data.html. Transform it into a pandas DataFrame and store it in an SQLite Database. 

In [24]:
# Imports
import sqlite3
import numpy as np
import pandas as pd

In [29]:
def transform_listings_data(df):
    
    # Drop variables with all there values missing
    df.dropna(axis=1, how='all', inplace=True)
    
    # Drop columns 
    df.drop(columns=['id', 'host_id', 'host_name', 'listing_url', 'last_scraped'], inplace=True)
    
    # Drop variables that are the same for every listing
    for col in df:
        if df[col].value_counts().shape[0] == 1:
            df.drop(columns=col, inplace=True)
            
    # Transform money columns from string:'$1,500.00'to float 1500.0
    money_cols = ['price', 'cleaning_fee', 'security_deposit']
    for col in money_cols:
        df[col] = df[col].dropna().apply(lambda p: float(p[1:].replace(",", "")))
    
    # Which ammenities to check for
    amenities = ["Doorman", "Gym", "Pool", "Wheelchair Accessible", "Fireplace", "AC",
                 "Washer/Dryer", "TV", "Breakfast", "Dog(s)", "Cat(s)", "24-Hour Check-in","Hot tub"]
    
    # Check to see if the listing offers these ammenities
    amenity_arr = np.array([df['amenities'].map(lambda amns: amn in amns) for amn in amenities])
    
    df = pd.concat([df, pd.DataFrame(data=amenity_arr.T, columns=amenities)], axis=1)
    

    return df

In [30]:
def load_listings_data(df, replace=True):
    
    conn = sqlite3.connect('data/airbnb.db')
    
    if replace:
        df.to_sql('listings', con = conn, if_exists='replace', index=False)
    else:
        df.to_sql('listings', con = conn, if_exists='append', index=False)
    conn.commit()
    conn.close()
    
    return None

In [31]:
# Execute this code cell to run the ETL pipeline

PATH_TO_DATA = 'data/New-York/listings.csv'      
city = 'New York'

# Extract

df = pd.read_csv(PATH_TO_DATA)
df_city['from_city'] = city

# Transform
cleaned_data = transform_listings_data(df)

# Load the Data
load_listings_data(cleaned_data)

In [32]:
conn = sqlite3.connect('data/airbnb.db')
pd.read_sql('SELECT * FROM listings', con = conn).shape

(50378, 104)